# Описание

Для [Следопыта по Воробьёвым горам](https://sledopyt-moscow.ru/competitions/98/rules_98.html) хотим подготовить специальный тэг. Текущие тэги позволяют только по районам выбирать дома, а там территория более специфическая.

# Импорты

In [40]:
import sys
import re
import os

import pandas as pd

sys.path.insert(1, '../service')

from search_engine import load_addresses, load_tags, DATA_PATH, SEP

# Данные

In [2]:
addresses_df = load_addresses()
_, ramenki_tags_df = load_tags(["Раменки"])
_, gagarinski_tags_df = load_tags(["Гагаринский"])

In [3]:
ramenki_tags_df.shape, gagarinski_tags_df.shape

((1822, 1), (561, 1))

In [4]:
ramenki_df = addresses_df.merge(ramenki_tags_df)
gagarinski_df = addresses_df.merge(gagarinski_tags_df)

ramenki_df["house_num_processed"] = ramenki_df["house_num"].apply(lambda x: int(re.findall("[0-9]+", x)[0]))
gagarinski_df["house_num_processed"] = gagarinski_df["house_num"].apply(lambda x: int(re.findall("[0-9]+", x)[0]))

# Удаление лишнего

## Раменки

In [5]:
ramenki_df.shape

(1820, 13)

In [6]:
sorted(ramenki_df["street_name"].unique())

['1-й Сетуньский',
 '2-й Мосфильмовский',
 '2-й Сетуньский',
 '3-й Сетуньский',
 '4-й Сетуньский',
 'Академика Хохлова',
 'Василия Ланового',
 'Вернадского',
 'Винницкая',
 'Воробьёвская',
 'Воробьёвское',
 'Генерала Дорохова',
 'Джавахарлала Неру',
 'Довженко',
 'Дружбы',
 'Косыгина',
 'Ленинские Горы',
 'Лобачевского',
 'Ломоносовский',
 'МЖД Киевское 5-й',
 'Менделеевская',
 'Мжд Киевское 3-й',
 'Минская',
 'Мичуринский',
 'Мосфильмовская',
 'Пудовкина',
 'Пырьева',
 'Раменки',
 'Раменский',
 'Светланова',
 'Сергея Бондарчука',
 'Столетова',
 'Удальцова',
 'Улофа Пальме',
 'Университетский',
 'Янковского']

In [7]:
ramenki_mask = True

Воробьёвское шоссе -- всё чётное, выкидываем.

In [8]:
ramenki_df[ramenki_df["street_name"] == "Воробьёвское"]["house_num"].unique()

array(['8', '4', '4А', '2', '6', '6А', '2Б', '2А'], dtype=object)

In [9]:
ramenki_mask &= ramenki_df["street_name"] != "Воробьёвское"
print(ramenki_mask.sum())

1786


Все улицы за Мосфильмовской -- выкидываем.

In [10]:
block_list_1 = [
    '1-й Сетуньский',
    '2-й Мосфильмовский',
    '2-й Сетуньский',
    '3-й Сетуньский',
    '4-й Сетуньский',
    'Довженко',
    'Пудовкина',
    'Пырьева',
    'Улофа Пальме',
    'Минская',
    'МЖД Киевское 5-й',
    'Мжд Киевское 3-й',
    'Генерала Дорохова',
    'Янковского',
    'Сергея Бондарчука',
    'Василия Ланового',
    'Винницкая',
]

In [11]:
ramenki_mask &= ~ramenki_df["street_name"].isin(block_list_1)
print(ramenki_mask.sum())

1275


Мосфильмовская -- остаётся только нечётная сторона до дома 51.

In [12]:
ramenki_mask &= ramenki_df.apply(
    lambda x: (x["street_name"] != 'Мосфильмовская') or ((x["house_num_processed"] % 2 == 1) and (x["house_num_processed"] < 53)),
    axis=1,
)
print(ramenki_mask.sum())

1127


У Столетова только чётная сторона.

In [13]:
ramenki_mask &= ramenki_df.apply(
    lambda x: (x["street_name"] != 'Столетова') or (x["house_num_processed"] % 2 == 0),
    axis=1,
)
print(ramenki_mask.sum())

1109


Мичуринский -- не больше 12 по чётной стороне и не больше 21 по нечётной. 

In [14]:
ramenki_mask &= ramenki_df.apply(
    lambda x: (x["street_name"] != 'Мичуринский')
    or ((x["house_num_processed"] % 2 == 0) and (x["house_num_processed"] <= 12))
    or ((x["house_num_processed"] % 2 == 1) and (x["house_num_processed"] <= 21)),
    axis=1,
)
print(ramenki_mask.sum())

980


Улицы к югу -- тоже выкидываем.

In [15]:
block_list_2 = [
    'Раменки',
    'Раменский',
    'Светланова',
    'Лобачевского',
    'Удальцова',
]

In [16]:
ramenki_mask &= ~ramenki_df["street_name"].isin(block_list_2)
print(ramenki_mask.sum())

698


С Вернадским не очень понятно, на всякий случай выкинем только 12 дом.

In [17]:
ramenki_mask &= ramenki_df.apply(
    lambda x: (x["street_name"] != 'Вернадского')
    or ((x["house_num_processed"] % 2 == 0) and (x["house_num_processed"] < 12)),
    axis=1,
)
print(ramenki_mask.sum())

677


In [18]:
sorted(ramenki_df[ramenki_mask]["street_name"].unique())

['Академика Хохлова',
 'Вернадского',
 'Воробьёвская',
 'Джавахарлала Неру',
 'Дружбы',
 'Косыгина',
 'Ленинские Горы',
 'Ломоносовский',
 'Менделеевская',
 'Мичуринский',
 'Мосфильмовская',
 'Столетова',
 'Университетский']

Согласно [Википедии](https://ru.wikipedia.org/wiki/Раменки_(район_Москвы)) в Раменках есть улицы (кроме удалённых нами):

Воробьёвская набережная
проспект Вернадского
Косыгина проспект
Ломоносовский проспект
Мичуринский проспект
Мосфильмовская улица
Университетский проспект

Прочие улицы:
Академика Самарского
Академика Хохлова
Дружбы
Колмогорова
Лебедева
Ленинские Горы
Менделеевская
Мичуринская аллея
Столетова
Шувалова

Площади:	
Джавахарлала Неру
Индиры Ганди
Университетская

## Гагаринский

In [19]:
gagarinski_df.shape

(561, 13)

In [20]:
sorted(gagarinski_df["street_name"].unique())

['60-летия Октября',
 'Академика Зелинского',
 'Академика Несмеянова',
 'Андреевская',
 'Бардина',
 'Вавилова',
 'Вернадского',
 'Губкина',
 'Дмитрия Ульянова',
 'Косыгина',
 'Ленинский',
 'Ломоносовский',
 'Молодёжная',
 'Панфёрова',
 'Строителей',
 'Университетский',
 'Фотиевой']

Нам нужно отбросить всё, что восточнее Ленинского проспекта.

In [21]:
gagarinski_mask = True

Ленинский проспект -- удалим нечётную сторону.

In [22]:
gagarinski_mask &= gagarinski_df.apply(
    lambda x: (x["street_name"] != 'Ленинский') or (x["house_num_processed"] % 2 == 0),
    axis=1,
)
print(gagarinski_mask.sum())

444


Часть улиц удаляем целиком.

In [23]:
block_list_3 = [
    'Панфёрова',
    'Вавилова',
    'Дмитрия Ульянова',
    'Губкина',
    'Бардина',
    'Академика Зелинского',
    'Академика Несмеянова',
    '60-летия Октября',
]

In [24]:
gagarinski_mask &= ~gagarinski_df["street_name"].isin(block_list_3)
print(gagarinski_mask.sum())

285


Наконец, обрежем Ломоносовский -- минимум дома 10 и 11.

In [25]:
gagarinski_mask &= gagarinski_df.apply(
    lambda x: (x["street_name"] != 'Ломоносовский') or (x["house_num_processed"] >= 10),
    axis=1,
)
print(gagarinski_mask.sum())

249


In [26]:
sorted(gagarinski_df[gagarinski_mask]["street_name"].unique())

['Андреевская',
 'Вернадского',
 'Косыгина',
 'Ленинский',
 'Ломоносовский',
 'Молодёжная',
 'Строителей',
 'Университетский',
 'Фотиевой']

Согласно [Википедии](https://ru.wikipedia.org/wiki/Гагаринский_район_(Москва)) в пересечении этого района и территории игры есть улицы:

Андреевская набережная
проспект Вернадского
4-й Воробьёвский проезд
Ездаков переулок
Ленинский проспект
Ломоносовский проспект
Университетский проспект

Прочие улицы:	
Академика Зельдовича
Анучина
Косыгина
Молодёжная
Николая Коперника
Сергея Капицы
Строителей
Фотиевой

Площади:	
60-летия СССР (на границе)
Академика Прохорова (на границе)
Академика Тамма
Гагарина (на границе)
Джавахарлала Неру

# Объединение

In [45]:
sparrow_hills_tags_df = pd.concat(
    [
        gagarinski_df[gagarinski_mask][["oid"]],
        ramenki_df[ramenki_mask][["oid"]],
    ],
    ignore_index=True,
)

In [46]:
sparrow_hills_tags_df["tag"] = "Следопыт98"

In [42]:
tags_path = os.path.join(DATA_PATH, "77_ht.tsv")

In [43]:
all_tags_df = pd.read_csv(tags_path, sep=SEP)

In [47]:
pd.concat(
    [
        all_tags_df,
        sparrow_hills_tags_df,
    ],
    ignore_index=True,
).to_csv(tags_path, sep=SEP, index=False)

# Список улиц

In [50]:
sorted(pd.concat(
    [
        gagarinski_df[gagarinski_mask]["street_name"],
        ramenki_df[ramenki_mask]["street_name"],
    ],
    ignore_index=True,
).unique())

['Академика Хохлова',
 'Андреевская',
 'Вернадского',
 'Воробьёвская',
 'Джавахарлала Неру',
 'Дружбы',
 'Косыгина',
 'Ленинские Горы',
 'Ленинский',
 'Ломоносовский',
 'Менделеевская',
 'Мичуринский',
 'Молодёжная',
 'Мосфильмовская',
 'Столетова',
 'Строителей',
 'Университетский',
 'Фотиевой']

In [ ]:
Не-улицы:
Андреевская набережная
проспект Вернадского
Воробьёвская набережная
Ездаков переулок (нет домов)
Косыгина проспект
Ленинский проспект
Ленинские Горы территория
Ломоносовский проспект
Мичуринская аллея (нет домов)
Мичуринский проспект
Университетский проспект
Четвёртый Воробьёвский проезд (нет домов)

Площади:	
60-летия СССР (на границе, нет домов)
Академика Прохорова (на границе, нет домов)
Академика Тамма (нет домов)
Гагарина (на границе, нет домов)
Джавахарлала Неру
Индиры Ганди (нет домов)
Университетская (нет домов)

Улицы:
Академика Зельдовича (только офис Яндекс, дом 1)
Академика Самарского (нет домов)
Академика Хохлова
Анучина (нет домов)
Дружбы 
Колмогорова (нет домов)
Лебедева (нет домов)
Менделеевская
Молодёжная
Мосфильмовская
Николая Коперника (нет домов)
Сергея Капицы (нет домов)
Столетова
Строителей
Фотиевой
Шувалова (нет домов)